# Compare All Experiment Results

This notebook provides a comprehensive comparison of all experiments.

**Author:** Gheith Alrawahi  
**Institution:** Nankai University

---


In [7]:
# Cell 1: Imports
import json
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

from config import RESULTS_DIR, ALL_EXPERIMENTS

print(f"Results directory: {RESULTS_DIR}")

# Try to import tabulate, install if not available
try:
    from tabulate import tabulate
except ImportError:
    print("Note: tabulate not installed. Using pandas display instead.")
    tabulate = None

Results directory: d:\Projects\KnowledgeDistillation\code_v2_32\results


In [8]:
# Cell 2: Load All Results (Updated for new structure with both teachers)
def find_experiment_dirs():
    """Find all experiment directories including timestamped ones."""
    experiments = {}
    
    for exp_dir in RESULTS_DIR.iterdir():
        if not exp_dir.is_dir():
            continue
        
        # Check if it has any final_results file
        has_results = any(exp_dir.glob('final_results*.json'))
        if not has_results:
            continue
        
        # Determine base experiment ID
        dir_name = exp_dir.name
        
        # Handle specific directories
        if dir_name == 'teacher':
            base_id = 'teacher'
        elif dir_name == 'teacher_v4':
            base_id = 'teacher_v4'
        else:
            # Handle timestamped directories (e.g., v1_baseline_20251207_135825)
            base_id = dir_name
            for known_id in ['v1_baseline', 'v2_enhanced', 'v3_dkd_beta8', 'v3_1_dkd_beta2', 'v4_saturation']:
                if dir_name.startswith(known_id):
                    base_id = known_id
                    break
        
        # If multiple runs exist, keep the most recent
        if base_id in experiments:
            existing_dir = experiments[base_id]['dir']
            if exp_dir.name > existing_dir.name:  # Newer timestamp
                experiments[base_id] = {'dir': exp_dir, 'name': dir_name}
        else:
            experiments[base_id] = {'dir': exp_dir, 'name': dir_name}
    
    return experiments

def load_all_results():
    """Load final results from all experiments."""
    exp_dirs = find_experiment_dirs()
    results = {}
    
    for base_id, info in exp_dirs.items():
        exp_dir = info['dir']
        
        # Find final_results file (check multiple possible names)
        final_path = None
        for fname in ['final_results.json', 'final_results_teacher.json', 'final_results_v4.json', 'final_results_teacher_v4.json']:
            candidate = exp_dir / fname
            if candidate.exists():
                final_path = candidate
                break
        
        if not final_path:
            continue
        
        with open(final_path, 'r') as f:
            data = json.load(f)
        
        # Also load config if available
        config_path = exp_dir / 'config.json'
        if config_path.exists():
            with open(config_path, 'r') as f:
                data['config'] = json.load(f)
        
        # Load history summary
        history_path = exp_dir / 'training_history.csv'
        if history_path.exists():
            data['history_available'] = True
            df = pd.read_csv(history_path)
            data['epochs_recorded'] = len(df)
        else:
            data['history_available'] = False
        
        data['dir_name'] = info['name']
        results[base_id] = data
    
    return results

all_results = load_all_results()
print(f"Loaded {len(all_results)} experiments:")
for exp_id, data in sorted(all_results.items()):
    dir_name = data.get('dir_name', exp_id)
    history = "✓" if data.get('history_available') else "✗"
    acc = data.get('results', {}).get('final_accuracy', 0)
    print(f"  - {exp_id} ({dir_name}) → {acc:.2f}% [history: {history}]")

Loaded 8 experiments:
  - final_results (final_results) → 76.65% [history: ✗]
  - teacher (teacher) → 76.65% [history: ✓]
  - teacher_v4 (teacher_v4) → 84.39% [history: ✓]
  - v1_baseline (v1_baseline_20251207_143632) → 76.12% [history: ✓]
  - v2_enhanced (v2_enhanced_20251208_021230) → 76.19% [history: ✓]
  - v3_1_dkd_beta2 (v3_1_dkd_beta2_20251208_132844) → 75.63% [history: ✓]
  - v3_dkd_beta8 (v3_dkd_beta8_20251208_104226) → 66.85% [history: ✓]
  - v4_saturation (v4_saturation_20251209_170910) → 77.93% [history: ✓]


In [9]:
# Cell 3: Create Comparison Table (Updated with both Teachers)
def get_value(data, *keys, default=0):
    """Safely get nested value from dict."""
    result = data
    for key in keys:
        if isinstance(result, dict):
            result = result.get(key, default)
        else:
            return default
    return result if result is not None else default

def create_comparison_table(results):
    """Create a detailed comparison table with both teachers."""
    rows = []
    
    # Define order - now includes both teachers
    order = ['teacher', 'teacher_v4', 'v1_baseline', 'v2_enhanced', 'v3_dkd_beta8', 'v3_1_dkd_beta2', 'v4_saturation']
    
    # Get teacher accuracies
    teacher_32_acc = get_value(results.get('teacher', {}), 'results', 'final_accuracy', default=0)
    teacher_64_acc = get_value(results.get('teacher_v4', {}), 'results', 'final_accuracy', default=0)
    
    for exp_id in order:
        if exp_id not in results:
            continue
        
        data = results[exp_id]
        r = data.get('results', {})
        c = data.get('config', {})
        d = c.get('distillation', {})
        a = c.get('augmentation', {})
        
        # Get accuracy (handle both old and new structure)
        accuracy = r.get('final_accuracy', r.get('student_accuracy', 0))
        
        # Determine model type and resolution
        if exp_id == 'teacher':
            model_type = 'teacher'
            resolution = '32x32'
        elif exp_id == 'teacher_v4':
            model_type = 'teacher'
            resolution = '64x64'
        else:
            model_type = 'student'
            resolution = '64→32' if exp_id == 'v4_saturation' else '32x32'
        
        # Calculate Teacher Gap for students (use appropriate teacher)
        if model_type == 'student':
            if exp_id == 'v4_saturation':
                # v4 uses the 64x64 teacher
                ref_teacher = teacher_64_acc if teacher_64_acc > 0 else teacher_32_acc
            else:
                # Other experiments use the 32x32 teacher
                ref_teacher = teacher_32_acc
            teacher_gap = ref_teacher - accuracy if ref_teacher > 0 else None
        else:
            teacher_gap = None
        
        row = {
            'ID': exp_id,
            'Type': model_type.capitalize(),
            'Resolution': resolution,
            'Method': d.get('method', '-') if model_type == 'student' else '-',
            'AutoAug': 'Y' if a.get('auto_augment') else 'N',
            'Accuracy (%)': accuracy,
            'Teacher Gap (%)': teacher_gap,
            'Best Epoch': r.get('best_epoch', 0),
            'Epochs': r.get('total_epochs', 0),
            'Time (min)': r.get('training_time_minutes', 0),
            'Early Stop': 'Y' if r.get('early_stopped') else 'N'
        }
        
        # Add retention for students
        if model_type == 'student':
            retention = r.get('retention_rate', 0)
            row['Retention (%)'] = retention if retention else 0
        else:
            row['Retention (%)'] = None
        
        rows.append(row)
    
    df = pd.DataFrame(rows)
    return df

comparison_df = create_comparison_table(all_results)

# Create display version with formatting
display_df = comparison_df.copy()
display_df['Accuracy (%)'] = display_df['Accuracy (%)'].apply(lambda x: f"{x:.2f}")
display_df['Teacher Gap (%)'] = display_df['Teacher Gap (%)'].apply(lambda x: f"{x:.2f}" if pd.notna(x) else '-')
display_df['Time (min)'] = display_df['Time (min)'].apply(lambda x: f"{x:.1f}")
display_df['Retention (%)'] = display_df['Retention (%)'].apply(lambda x: f"{x:.1f}" if pd.notna(x) and x > 0 else '-')

print("\n" + "=" * 120)
print("EXPERIMENT COMPARISON (with Both Teachers)")
print("=" * 120)

if tabulate:
    print(tabulate(display_df, headers='keys', tablefmt='grid', showindex=False))
else:
    print(display_df.to_string(index=False))


EXPERIMENT COMPARISON (with Both Teachers)
+----------------+---------+--------------+-------------+-----------+----------------+-------------------+--------------+----------+--------------+--------------+-----------------+
| ID             | Type    | Resolution   | Method      | AutoAug   |   Accuracy (%) | Teacher Gap (%)   |   Best Epoch |   Epochs |   Time (min) | Early Stop   | Retention (%)   |
+================+=========+==============+=============+===========+================+===================+==============+==========+==============+==============+=================+
| teacher        | Teacher | 32x32        | -           | N         |          76.65 | -                 |          199 |      200 |        389.7 | N            | -               |
+----------------+---------+--------------+-------------+-----------+----------------+-------------------+--------------+----------+--------------+--------------+-----------------+
| teacher_v4     | Teacher | 64x64        | -      

In [10]:
# Cell 4: Key Findings (Updated with both Teachers)
print("\n" + "=" * 80)
print("KEY FINDINGS - SURGICAL ANALYSIS")
print("=" * 80)

# Get both teacher accuracies
teacher_32_acc = get_value(all_results.get('teacher', {}), 'results', 'final_accuracy', default=0)
teacher_64_acc = get_value(all_results.get('teacher_v4', {}), 'results', 'final_accuracy', default=0)

print(f"\n📊 TEACHER MODELS:")
print(f"   Teacher 32x32 (EfficientNetV2-L): {teacher_32_acc:.2f}%")
print(f"   Teacher 64x64 (EfficientNetV2-L): {teacher_64_acc:.2f}%")
print(f"   Resolution Gain: +{teacher_64_acc - teacher_32_acc:.2f}%")

# Get all student accuracies
v1_acc = get_value(all_results.get('v1_baseline', {}), 'results', 'final_accuracy', default=0)
v2_acc = get_value(all_results.get('v2_enhanced', {}), 'results', 'final_accuracy', default=0)
v3_acc = get_value(all_results.get('v3_dkd_beta8', {}), 'results', 'final_accuracy', default=0)
v3_1_acc = get_value(all_results.get('v3_1_dkd_beta2', {}), 'results', 'final_accuracy', default=0)
v4_acc = get_value(all_results.get('v4_saturation', {}), 'results', 'final_accuracy', default=0)

# Finding 1: Robustness of Standard KD
print(f"\n" + "-" * 80)
print("🔬 FINDING 1: Robustness of Standard KD")
print("-" * 80)
print(f"""
   Standard KD proved exceptionally robust against strong augmentation.
   
   • Baseline (v1, no AutoAug): {v1_acc:.2f}%
   • Enhanced (v2, with AutoAug): {v2_acc:.2f}%
   • Improvement: +{v2_acc - v1_acc:.2f}%
   
   ➜ The marginal gain (+{v2_acc - v1_acc:.2f}%) despite adding AutoAugment indicates
     that Standard KD is highly DATA-EFFICIENT and STABLE.
""")

# Finding 2: Fragility of High-Beta DKD
print("-" * 80)
print("⚠️ FINDING 2: Fragility of High-Beta DKD (Regularization-Distillation Conflict)")
print("-" * 80)
print(f"""
   Experiment v3 (DKD, β=8.0) collapsed dramatically.
   
   • v3 (DKD β=8.0): {v3_acc:.2f}% ← COLLAPSED (Early stopped at epoch 84)
   • v3.1 (DKD β=2.0): {v3_1_acc:.2f}% ← RECOVERED
   • Recovery gain: +{v3_1_acc - v3_acc:.2f}%
   
   ➜ High reliance on dark knowledge (β=8.0) interferes with strong augmentation noise.
   ➜ DKD requires SENSITIVE TUNING compared to Standard KD.
""")

# Finding 3: Capacity Saturation (Most Important)
print("-" * 80)
print("🎯 FINDING 3: Capacity Saturation Analysis (KEY RESULT)")
print("-" * 80)
v2_gap = teacher_32_acc - v2_acc if teacher_32_acc > 0 else 0
v4_gap = teacher_64_acc - v4_acc if teacher_64_acc > 0 else 0
teacher_improvement = teacher_64_acc - teacher_32_acc
student_improvement = v4_acc - v2_acc
efficiency = (student_improvement / teacher_improvement * 100) if teacher_improvement > 0 else 0

print(f"""
   Comparison between v2 (32x32 teacher) and v4 (64x64 teacher):
   
   TEACHERS:
   • Teacher 32x32: {teacher_32_acc:.2f}%
   • Teacher 64x64: {teacher_64_acc:.2f}% (+{teacher_improvement:.2f}%)
   
   STUDENTS:
   • v2 Student: {v2_acc:.2f}% (Gap: {v2_gap:.2f}%)
   • v4 Student: {v4_acc:.2f}% (Gap: {v4_gap:.2f}%)
   
   ANALYSIS:
   • Teacher improved by: +{teacher_improvement:.2f}%
   • Student improved by: +{student_improvement:.2f}%
   • Knowledge Transfer Efficiency: {efficiency:.1f}%
   
   ➜ The student captured only {efficiency:.1f}% of the teacher's improvement.
   ➜ This demonstrates PARTIAL CAPACITY SATURATION - the student benefits from
     a stronger teacher but with DIMINISHING RETURNS.
""")

# Summary Statistics
print("=" * 80)
print("📈 SUMMARY STATISTICS")
print("=" * 80)
best_student = max(v1_acc, v2_acc, v3_acc, v3_1_acc, v4_acc)
print(f"""
   Best Student Accuracy:     {best_student:.2f}% (v4)
   Best Retention (vs 32x32): {v2_acc/teacher_32_acc*100:.2f}% (v2)
   Best Retention (vs 64x64): {v4_acc/teacher_64_acc*100:.2f}% (v4)
   
   Teacher Gap (v2 vs 32x32): {v2_gap:.2f}%
   Teacher Gap (v4 vs 64x64): {v4_gap:.2f}%
   
   Standard KD vs DKD Winner: Standard KD (+{best_student - v3_1_acc:.2f}%)
""")


KEY FINDINGS - SURGICAL ANALYSIS

📊 TEACHER MODELS:
   Teacher 32x32 (EfficientNetV2-L): 76.65%
   Teacher 64x64 (EfficientNetV2-L): 84.39%
   Resolution Gain: +7.74%

--------------------------------------------------------------------------------
🔬 FINDING 1: Robustness of Standard KD
--------------------------------------------------------------------------------

   Standard KD proved exceptionally robust against strong augmentation.
   
   • Baseline (v1, no AutoAug): 76.12%
   • Enhanced (v2, with AutoAug): 76.19%
   • Improvement: +0.07%
   
   ➜ The marginal gain (+0.07%) despite adding AutoAugment indicates
     that Standard KD is highly DATA-EFFICIENT and STABLE.

--------------------------------------------------------------------------------
⚠️ FINDING 2: Fragility of High-Beta DKD (Regularization-Distillation Conflict)
--------------------------------------------------------------------------------

   Experiment v3 (DKD, β=8.0) collapsed dramatically.
   
   • v3 (DKD β

In [11]:
# Cell 5: Export Publication-Quality LaTeX Table (Updated with both Teachers)
def export_professional_latex_table(df, filename):
    """Export a publication-quality LaTeX table with auto-bolded best results."""
    
    # Get teacher accuracies for caption
    teacher_32_acc = get_value(all_results.get('teacher', {}), 'results', 'final_accuracy', default=0)
    teacher_64_acc = get_value(all_results.get('teacher_v4', {}), 'results', 'final_accuracy', default=0)
    
    # Prepare data for LaTeX - include both teachers and students
    latex_df = df.copy()
    
    # Get max/min values for highlighting (students only)
    student_df = latex_df[latex_df['Type'] == 'Student']
    max_accuracy = student_df['Accuracy (%)'].max() if len(student_df) > 0 else 0
    min_gap = student_df['Teacher Gap (%)'].min() if len(student_df) > 0 else 0
    
    # Build LaTeX table body
    latex_body = ""
    
    for _, row in latex_df.iterrows():
        exp_id = row['ID']
        model_type = row['Type']
        
        # Format experiment ID
        if exp_id == 'teacher':
            exp_display = 'Teacher (32×32)'
        elif exp_id == 'teacher_v4':
            exp_display = 'Teacher (64×64)'
        else:
            exp_display = exp_id.replace('_', '-')
        
        # Format method name
        method = row['Method']
        if method == 'standard_kd':
            method_display = 'Standard KD'
        elif method == 'dkd':
            if 'beta8' in exp_id:
                method_display = 'DKD ($\\beta=8.0$)'
            elif 'beta2' in exp_id:
                method_display = 'DKD ($\\beta=2.0$)'
            else:
                method_display = 'DKD'
        else:
            method_display = '-'
        
        # Format resolution
        resolution = row['Resolution']
        
        # Format accuracy (bold if max for students)
        acc = row['Accuracy (%)']
        if model_type == 'Student' and acc == max_accuracy:
            acc_str = f"\\textbf{{{acc:.2f}}}"
        else:
            acc_str = f"{acc:.2f}"
        
        # Format teacher gap (bold if min - smaller is better)
        gap = row['Teacher Gap (%)']
        if pd.notna(gap):
            if model_type == 'Student' and gap == min_gap:
                gap_str = f"\\textbf{{{gap:.2f}}}"
            else:
                gap_str = f"{gap:.2f}"
        else:
            gap_str = '-'
        
        # Format time
        time_str = f"{row['Time (min)']:.1f}"
        
        # AutoAug indicator
        autoaug = "\\checkmark" if row['AutoAug'] == 'Y' else "-"
        
        # Add horizontal line before students
        if exp_id == 'v1_baseline':
            latex_body += "\\midrule\n"
        
        latex_body += f"    {exp_display} & {resolution} & {method_display} & {autoaug} & {acc_str} & {gap_str} & {time_str} \\\\\n"
    
    # Complete LaTeX table
    latex_table = f"""% Auto-generated LaTeX table for thesis
% Best student results are highlighted in \\textbf{{bold}}

\\begin{{table}}[htbp]
\\centering
\\caption{{Comparison of Knowledge Distillation Methods on CIFAR-100. 
Teachers: EfficientNetV2-L at 32×32 ({teacher_32_acc:.2f}\\%) and 64×64 ({teacher_64_acc:.2f}\\%). 
Student: EfficientNetV2-S. Best student results are highlighted in \\textbf{{bold}}.}}
\\label{{tab:kd_comparison}}
\\begin{{tabular}}{{llccccc}}
\\toprule
\\textbf{{Experiment}} & \\textbf{{Resolution}} & \\textbf{{Method}} & \\textbf{{AutoAug}} & \\textbf{{Accuracy}} & \\textbf{{Gap}} & \\textbf{{Time}} \\\\
 & & & & \\textbf{{(\\%)}} & \\textbf{{(\\%)}} & \\textbf{{(min)}} \\\\
\\midrule
{latex_body}\\bottomrule
\\end{{tabular}}
\\end{{table}}
"""
    
    # Save to file
    output_path = RESULTS_DIR / filename
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(latex_table)
    
    print(f"✓ Publication-quality LaTeX table saved: {output_path}")
    print("\n" + "=" * 80)
    print("LATEX TABLE PREVIEW:")
    print("=" * 80)
    print(latex_table)
    
    return latex_table

# Generate the professional LaTeX table
latex_output = export_professional_latex_table(comparison_df, 'comparison_table_professional.tex')

# Also save the simple version
display_df.to_csv(RESULTS_DIR / 'comparison_table.csv', index=False)
print(f"\n✓ CSV saved: {RESULTS_DIR / 'comparison_table.csv'}")

✓ Publication-quality LaTeX table saved: d:\Projects\KnowledgeDistillation\code_v2_32\results\comparison_table_professional.tex

LATEX TABLE PREVIEW:
% Auto-generated LaTeX table for thesis
% Best student results are highlighted in \textbf{bold}

\begin{table}[htbp]
\centering
\caption{Comparison of Knowledge Distillation Methods on CIFAR-100. 
Teachers: EfficientNetV2-L at 32×32 (76.65\%) and 64×64 (84.39\%). 
Student: EfficientNetV2-S. Best student results are highlighted in \textbf{bold}.}
\label{tab:kd_comparison}
\begin{tabular}{llccccc}
\toprule
\textbf{Experiment} & \textbf{Resolution} & \textbf{Method} & \textbf{AutoAug} & \textbf{Accuracy} & \textbf{Gap} & \textbf{Time} \\
 & & & & \textbf{(\%)} & \textbf{(\%)} & \textbf{(min)} \\
\midrule
    Teacher (32×32) & 32x32 & - & - & 76.65 & - & 389.7 \\
    Teacher (64×64) & 64x64 & - & - & 84.39 & - & 1048.9 \\
\midrule
    v1-baseline & 32x32 & Standard KD & - & 76.12 & 0.53 & 300.2 \\
    v2-enhanced & 32x32 & Standard KD & \chec

In [12]:
# Cell 6: Generate Enhanced Summary Report (Updated with both Teachers)
def generate_enhanced_summary_report():
    """Generate a publication-quality markdown summary report with both teachers."""
    
    # Get key values
    teacher_32_acc = get_value(all_results.get('teacher', {}), 'results', 'final_accuracy', default=0)
    teacher_64_acc = get_value(all_results.get('teacher_v4', {}), 'results', 'final_accuracy', default=0)
    v1_acc = get_value(all_results.get('v1_baseline', {}), 'results', 'final_accuracy', default=0)
    v2_acc = get_value(all_results.get('v2_enhanced', {}), 'results', 'final_accuracy', default=0)
    v3_acc = get_value(all_results.get('v3_dkd_beta8', {}), 'results', 'final_accuracy', default=0)
    v3_1_acc = get_value(all_results.get('v3_1_dkd_beta2', {}), 'results', 'final_accuracy', default=0)
    v4_acc = get_value(all_results.get('v4_saturation', {}), 'results', 'final_accuracy', default=0)
    
    report = []
    report.append("# Experiment Results Summary\n")
    report.append(f"**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M')}\n")
    
    report.append("\n## Overview\n")
    report.append(f"- **Total experiments:** {len(all_results)}")
    report.append(f"- **Teacher model (32x32):** EfficientNetV2-L ({teacher_32_acc:.2f}%)")
    report.append(f"- **Teacher model (64x64):** EfficientNetV2-L ({teacher_64_acc:.2f}%)")
    report.append(f"- **Student model:** EfficientNetV2-S")
    report.append(f"- **Dataset:** CIFAR-100")
    report.append(f"- **Training epochs:** 200 (with early stopping)\n")
    
    report.append("\n## Results Table\n")
    report.append(display_df.to_markdown(index=False))
    report.append("\n")
    
    # Key Findings with Refined Narrative
    report.append("\n## Key Findings\n")
    
    report.append("\n### 1. Higher Resolution Improves Teacher\n")
    teacher_gain = teacher_64_acc - teacher_32_acc
    report.append(f"""Training the teacher at 64×64 resolution instead of 32×32 improved accuracy 
from {teacher_32_acc:.2f}% to {teacher_64_acc:.2f}% (**+{teacher_gain:.2f}%**). This demonstrates 
that higher resolution provides more visual information for the model to learn from.\n""")
    
    report.append("\n### 2. Robustness of Standard KD\n")
    report.append(f"""Standard KD proved exceptionally robust against strong augmentation. 
The baseline (v1) achieved {v1_acc:.2f}%, and the enhanced version (v2) reached {v2_acc:.2f}%. 
The marginal gain (+{v2_acc - v1_acc:.2f}%) despite adding AutoAugment indicates that 
**Standard KD is highly data-efficient** and stable.\n""")
    
    report.append("\n### 3. Fragility of High-Beta DKD\n")
    report.append(f"""Experiment v3 (DKD, β=8.0) collapsed to {v3_acc:.2f}%, confirming the 
**Regularization-Distillation Conflict**. High reliance on dark knowledge interferes with 
strong augmentation noise. Reducing β to 2.0 (v3.1) recovered performance to {v3_1_acc:.2f}%, 
validating that DKD requires sensitive tuning compared to Standard KD.\n""")
    
    report.append("\n### 4. Capacity Saturation Analysis (Key Result)\n")
    v2_gap = teacher_32_acc - v2_acc
    v4_gap = teacher_64_acc - v4_acc
    student_gain = v4_acc - v2_acc
    efficiency = (student_gain / teacher_gain * 100) if teacher_gain > 0 else 0
    
    report.append(f"""The most significant finding is the comparison between v2 and v4:

| Metric | v2 (32×32 Teacher) | v4 (64×64 Teacher) |
|--------|-------------------|-------------------|
| Teacher Accuracy | {teacher_32_acc:.2f}% | {teacher_64_acc:.2f}% |
| Student Accuracy | {v2_acc:.2f}% | {v4_acc:.2f}% |
| Teacher Gap | {v2_gap:.2f}% | {v4_gap:.2f}% |

**Analysis:**
- Teacher improved by: **+{teacher_gain:.2f}%**
- Student improved by: **+{student_gain:.2f}%**
- Knowledge Transfer Efficiency: **{efficiency:.1f}%**

The student captured only {efficiency:.1f}% of the teacher's improvement, demonstrating 
**partial capacity saturation** - the student benefits from a stronger teacher but with 
**diminishing returns**.\n""")
    
    report.append("\n## Summary Statistics\n")
    best_acc = max(v1_acc, v2_acc, v3_acc, v3_1_acc, v4_acc)
    report.append(f"| Metric | Value |")
    report.append(f"|--------|-------|")
    report.append(f"| Best Student Accuracy | {best_acc:.2f}% |")
    report.append(f"| Best Retention (vs 32×32) | {v2_acc/teacher_32_acc*100:.2f}% |")
    report.append(f"| Best Retention (vs 64×64) | {v4_acc/teacher_64_acc*100:.2f}% |")
    report.append(f"| Teacher Gap (v2) | {v2_gap:.2f}% |")
    report.append(f"| Teacher Gap (v4) | {v4_gap:.2f}% |")
    report.append(f"| Knowledge Transfer Efficiency | {efficiency:.1f}% |")
    report.append("")
    
    report.append("\n## Experiment Details\n")
    exp_order = ['teacher', 'teacher_v4', 'v1_baseline', 'v2_enhanced', 'v3_dkd_beta8', 'v3_1_dkd_beta2', 'v4_saturation']
    for exp_id in exp_order:
        if exp_id not in all_results:
            continue
        data = all_results[exp_id]
        results = data.get('results', {})
        acc = get_value(data, 'results', 'final_accuracy', default=0)
        
        # Display name
        if exp_id == 'teacher':
            display_name = 'Teacher (32×32)'
        elif exp_id == 'teacher_v4':
            display_name = 'Teacher (64×64)'
        else:
            display_name = exp_id
        
        report.append(f"\n### {display_name}")
        report.append(f"- **Accuracy:** {acc:.2f}%")
        report.append(f"- **Best Epoch:** {results.get('best_epoch', 'N/A')}")
        report.append(f"- **Training Time:** {results.get('training_time_minutes', 0):.1f} minutes")
        
        # Teacher gap for students
        if exp_id not in ['teacher', 'teacher_v4']:
            if exp_id == 'v4_saturation':
                gap = teacher_64_acc - acc
                report.append(f"- **Teacher Gap (vs 64×64):** {gap:.2f}%")
            else:
                gap = teacher_32_acc - acc
                report.append(f"- **Teacher Gap (vs 32×32):** {gap:.2f}%")
        report.append("")
    
    # Write report
    report_text = '\n'.join(report)
    report_path = RESULTS_DIR / 'summary_report.md'
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write(report_text)
    
    print(f"✓ Enhanced summary report saved: {report_path}")
    return report_text

report = generate_enhanced_summary_report()
print("\n" + "=" * 80)
print(report)

✓ Enhanced summary report saved: d:\Projects\KnowledgeDistillation\code_v2_32\results\summary_report.md

# Experiment Results Summary

**Generated:** 2025-12-10 16:45


## Overview

- **Total experiments:** 8
- **Teacher model (32x32):** EfficientNetV2-L (76.65%)
- **Teacher model (64x64):** EfficientNetV2-L (84.39%)
- **Student model:** EfficientNetV2-S
- **Dataset:** CIFAR-100
- **Training epochs:** 200 (with early stopping)


## Results Table

| ID             | Type    | Resolution   | Method      | AutoAug   |   Accuracy (%) | Teacher Gap (%)   |   Best Epoch |   Epochs |   Time (min) | Early Stop   | Retention (%)   |
|:---------------|:--------|:-------------|:------------|:----------|---------------:|:------------------|-------------:|---------:|-------------:|:-------------|:----------------|
| teacher        | Teacher | 32x32        | -           | N         |          76.65 | -                 |          199 |      200 |        389.7 | N            | -               |
| teac